In [1]:
import syft as sy
import torch as t
import numpy as np
import pandas as pd

## Launch a Duet Sever

In [2]:
duet = sy.launch_duet(loopback = True)

🎤  🎸  ♪♪♪ Starting Duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently in beta.
♫♫♫ > Use at your own risk.


    > ❤️ Love Duet? Please consider supporting our community!
    > https://github.com/sponsors/OpenMined

♫♫♫ > Punching through firewall to OpenGrid Network Node at:
♫♫♫ > http://ec2-18-218-7-180.us-east-2.compute.amazonaws.com:5000
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... 
♫♫♫ > DONE!

♫♫♫ > STEP 1: Send the following code to your Duet Partner!

import syft as sy
duet = sy.join_duet(loopback=True)

♫♫♫ > Connecting...

♫♫♫ > CONNECTED!



## Read Data and Preprocess

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

german = pd.read_csv("./data/german_data_1.csv")  

In [ ]:
test_size = 0.20

processed_data = None
categorical = None
label_encoders = {}

def preprocessing(dataset, data, test_size):
    """
    Preprocess dataset

    Parameters
    ----------
    data: DataFrame
        Pandas dataframe containing German dataset.
    """
    
    global processed_data
    global categorical
    global label_encoders

    # Reset global variables
    
    processed_data = None
    categorical = None
    label_encoders = {}


    if dataset == "German":
        # Drop savings account and checkings account columns as they contain a lot
        # of NaN values and may not always be available in real life scenarios
        data = data.drop(columns = ['Saving accounts', 'Checking account'])
        
    dat_dict = data.to_dict()
    new_dat_dict = {}

    # rename columns(Make them lowercase and snakecase)
    for key, value in dat_dict.items():
        newKey = key
        if type(key) == str:
            newKey = newKey.lower().replace(' ', '_')
        # if newKey != key:
        new_dat_dict[newKey] = dat_dict[key]
    del dat_dict

    data = pd.DataFrame.from_dict(new_dat_dict)
    del new_dat_dict


    # print(data.describe())
    # print(data.describe(include='O'))

    cols = data.columns
    num_cols = data._get_numeric_data().columns
    categorical = list(set(cols) - set(num_cols))

    # Drop null rows
    data = data.dropna()

    # Encode text columns to number values
    for category in categorical:
        le = LabelEncoder()
        data[category] = le.fit_transform(data[category])
        label_encoders[category] = le

    for col in data.columns:
        if(col not in categorical):
            data[col] = (data[col].astype('float') - np.mean(data[col].astype('float')))/np.std(data[col].astype('float'))

    # print(data.describe())
    # print(data.describe(include='O'))

    processed_data = data

    # Get Training parameters
    if dataset == "German":
        target_col = data.columns[-1]
        x = data.drop(columns=target_col, axis=1)
        y = data[target_col].astype('int')
    elif dataset == "Australian":
        x = data.drop(14, axis=1)
        y = data[14].astype('int')
    elif dataset == "Japanese":
        x = data.drop(15, axis=1)
        y = data[15].astype('int')
    elif dataset == "Taiwan":
        x = data.drop('default_payment_next_month', axis=1)
        y = data['default_payment_next_month'].astype('int')
    elif dataset == "Polish":
        x = data.drop('class', axis=1)
        y = data['class'].astype('int')


    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = test_size)
    x_train = pd.DataFrame(x_train)
    y_train = pd.DataFrame(y_train)
    y_test = pd.DataFrame(y_test)

    sc = StandardScaler()
    x_train = sc.fit_transform(x_train)
    x_test = sc.transform(x_test)
    
    y_train = y_train[y_train.columns[0]].to_numpy()
    y_test = y_test[y_test.columns[0]].to_numpy()

    return (x_train, x_test, y_train, y_test)

X_train, X_test, y_train, y_test = preprocessing("German", german, test_size)

In [ ]:
# Trnasform the input to tensor
X_train_tensor = t.FloatTensor(X_train)
y_train_tensor = t.tensor(y_train,dtype=t.long)
X_test_tensor = t.FloatTensor(X_test)
y_test_tensor = t.tensor(y_test,dtype=t.long)

## Send data to Duet server

In [ ]:
X_train_tensor.tag("X_Train_Owner1")
X_train_tensor.describe("Dataset of 400 samples, 7 feature")
X_train_ptr = X_train_tensor.send(duet,pointable = True)

y_train_tensor.tag("y_Train_Owner1")
y_train_tensor.describe("Dataset of 400 samples, 1 feature")
y_train_ptr = y_train_tensor.send(duet,pointable = True)

X_test_tensor.tag("X_Test_Owner1")
X_test_tensor.describe("Dataset of 400 samples, 7 feature")
X_test_ptr = X_test_tensor.send(duet,pointable = True)

y_test_tensor.tag("y_Test_Owner1")
y_test_tensor.describe("Dataset of 400 samples, 1 feature")
y_test_ptr = y_test_tensor.send(duet,pointable = True)


## Create a request handler

In [ ]:
duet.requests.add_handler(
    action="accept",
    print_local=True,  # print the result in your notebook
)

## Build a local Classifier

In [ ]:
class Net(t.nn.Module):
    def __init__(self,input_channels,output_channels):
        super(Net,self).__init__()
        #Our network:
        # Linear1->relu->Batchnorm->Linear2->relu->Batchnorm->Dropout->Linear3->output
        # Softmax is added in the predict function
        #This applies Linear transformation to input data. 
        self.fc1 = t.nn.Linear(input_channels,int(1.5*input_channels))
        self.fc2 = t.nn.Linear(int(1.5*input_channels),int(1.5*input_channels))
        self.fc3 = t.nn.Linear(int(1.5*input_channels),output_channels)
        
        self.relu = t.nn.ReLU()
        self.dropout = t.nn.Dropout(p=0.1)
        self.batchnorm1 = t.nn.BatchNorm1d(int(1.5*input_channels))
        self.batchnorm2 = t.nn.BatchNorm1d(int(1.5*input_channels))
        self.sigmoid = t.nn.Sigmoid()
    #This must be implemented
    def forward(self,x):
        x = self.relu(self.fc1(x))
        x = self.batchnorm1(x)
        x = self.relu(self.fc2(x))
        x = self.batchnorm2(x)
        x = self.dropout(x)
        x = self.fc3(x)
        x = self.sigmoid(x)
        return x
    def predict(self,x):
        output = self.forward(x)
        prediction = t.argmax(output,1)
        return prediction
        


In [ ]:
def classifier_train(epochs, model, optimizer, X,y,criterion):
    losses = []
    for i in range(epochs):
        #Precit the output for Given input
        y_pred = model.forward(X)
        #Compute Cross entropy loss
        loss = criterion(y_pred,y)
        #Add loss to the list
        losses.append(loss.item())
        
        #Print loss
        if i%500==0:
            print("Epoch:",i," Loss:",loss.item())
        
        #Clear the previous gradients
        optimizer.zero_grad()
        #Compute gradients
        loss.backward()
        #Adjust weights
        optimizer.step()
    return losses


## Local Training

In [ ]:
# Define the input and output
input_channels = X_train_tensor.shape[1]
output_channels = 2

#Initialize the model        
model = Net(input_channels,output_channels)
#Define loss criterion
criterion = t.nn.CrossEntropyLoss()
#Define the optimizer
optimizer = t.optim.Adam(model.parameters(), lr=0.001)
#Number of epochs
epochs = 5000

losses = classifier_train(
    epochs, model, optimizer,X_train_tensor,y_train_tensor,criterion
)

## Accuracy on test set

In [ ]:
from sklearn.metrics import accuracy_score

def accuracy(model,X,y):
    print(accuracy_score(model.predict(X),y))

In [ ]:
accuracy(model,X_test_tensor,y_test_tensor)